In [2]:
import os
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [3]:
# Laden der Umgebungsvariablen aus der .env-Datei
load_dotenv()
uri = os.environ['DB_URI']

# Herstellen der Verbindung zur MongoDB-Datenbank
client = MongoClient(uri)
db = client["rosen"]
collection = db["big_data_test"]

In [4]:
# Funktion zum Ersetzen von NaN-Werten
def replace_nan(values, attribute):
    new_values = []
    changed = False  # Flag, um zu überprüfen, ob Werte geändert wurden
    
    if attribute == 'distance':
        # Durchführung einer linearen Regression
        x = np.arange(len(values))
        valid_indices = ~np.isnan(values)
        slope, intercept, _, _, _ = linregress(x[valid_indices], values[valid_indices])
        # Ersetzen von NaN-Werten durch den linearen Wert
        new_values = [slope * i + intercept if pd.isna(val) else val for i, val in enumerate(values)]
        changed = True
    elif attribute == 'timestamp':
        # Bereinigung von NaN-Werten im Zeitstempel
        new_values = clean_timestamp(values)
        changed = True
    elif attribute == 'velocity':
        # Bereinigung von NaN-Werten in der Geschwindigkeit
        new_values = clean_velocity(values)
        changed = True
    elif attribute in ['magnetization', 'wall_thickness']:
        # Bereinigung von NaN-Werten in Magnetisierung und Wanddicke
        new_values = clean_average(values)
        changed = True
    else:
        # Wenn das Attribut nicht 'distance', 'timestamp', 'velocity', 'magnetization' oder 'wall_thickness' ist, bleiben die Werte unverändert
        new_values = values
        
    return new_values, changed

# Funktion zur Bereinigung von NaN-Werten im Zeitstempel
def clean_timestamp(values):
    new_values = values.copy()
    for i, val in enumerate(values):
        if pd.isna(val):
            if i == 0:
                # Bereinigung des Anfangswerts: Regression mit den nächsten 20 Werten
                new_values[i] = np.nanmean(values[1:21])
            elif i == len(values) - 1:
                # Bereinigung des Endwerts: Regression mit den vorherigen 20 Werten
                new_values[i] = np.nanmean(values[-21:-1])
            else:
                # Bereinigung in der Mitte: Linienregression mit den 10 Werten davor und danach
                start = max(0, i - 10)
                end = min(len(values), i + 11)
                valid_values = [v for v in values[start:end] if not pd.isna(v)]
                if valid_values:
                    x = np.arange(len(valid_values))
                    slope, intercept, _, _, _ = linregress(x, valid_values)
                    new_values[i] = slope * (i - start) + intercept
                else:
                    new_values[i] = np.nan
    return new_values

# Funktion zur Bereinigung von NaN-Werten in der Geschwindigkeit
def clean_velocity(values):
    new_values = values.copy()
    valid_values = [v for v in values if not pd.isna(v)]
    if len(valid_values) >= 11:
        for i, val in enumerate(values):
            if pd.isna(val):
                start = max(0, i - 5)
                end = min(len(values), i + 6)
                valid_values = [v for v in values[start:end] if not pd.isna(v)]
                try:
                    new_values[i] = np.nanmean(valid_values)
                except:
                    print('error' +values['_id'])
    else:
        mean_value = np.nanmean(values)
        new_values = [mean_value if pd.isna(val) else val for val in values]
    return new_values

# Funktion zur Bereinigung von NaN-Werten in Magnetisierung und Wanddicke
def clean_average(values):
    new_values = values.copy()
    valid_values = [v for v in values if not pd.isna(v)]
    if len(valid_values) >= 11:
        for i, val in enumerate(values):
            if pd.isna(val):
                start = max(0, i - 5)
                end = min(len(values), i + 6)
                valid_values = [v for v in values[start:end] if not pd.isna(v)]
                if len(valid_values) >= 6:  # Überprüfen, ob mindestens 6 gültige Werte vorhanden sind, um den Durchschnitt zu berechnen
                    new_values[i] = np.nanmean(valid_values)
                else:
                    new_values[i] = np.nan  # Setze NaN, wenn nicht genügend gültige Werte vorhanden sind
    else:
        mean_value = np.nanmean(values)
        new_values = [mean_value if pd.isna(val) else val for val in values]
    return new_values
# Abrufen der Dokumente aus der MongoDB-Sammlung
documents = collection.find({})

# Liste für gefundene IDs
found_ids = []
data_sets = []

# Iterieren über jedes Dokument
for document in documents:
    # Die Datenstruktur hat sich geändert, daher müssen wir die entsprechenden Schlüssel verwenden
    measuring_points = document
    has_nan = False
    for key, values in measuring_points.items():
        # Überprüfen, ob NaN-Werte in den Werten vorhanden sind
        if values is not None and any(pd.isna(val) for val in values):
            has_nan = True
            # Ersetzen von NaN-Werten entsprechend dem Attribut
            new_values, changed = replace_nan(values, key)
            if changed:
                measuring_points[key] = new_values
    
    if has_nan:
        # Aktualisieren des Dokuments in der Datenbank
        # collection.update_one({"_id": document["_id"]}, {"$set": {"measuring_points": measuring_points}})
        # Hinzufügen der ID zum DataFrame
        found_ids.append(document['_id'])
        data_sets.append(measuring_points) 

C:\Users\golde\AppData\Local\Temp\ipykernel_11664\4030345237.py:67: RuntimeWarning: Mean of empty slice
  new_values[i] = np.nanmean(valid_values)


In [15]:
# Erstellen des DataFrame
df = pd.DataFrame({
    "_id": found_ids,
    "measuring_points": data_sets
})

# Ausgabe des DataFrames
print(df)

                                      _id  \
0    007ff213-9b5d-4243-9c8b-5eae997ac0ee   
1    0108e474-f1f4-4274-bf9f-e51d4963c091   
2    023794f2-5e3d-4450-8b71-8aae79d381a1   
3    0245cbf0-faec-4708-a1f2-3ccf7ebec1e1   
4    02498907-b394-4b7c-a0ab-068692746591   
..                                    ...   
790  fe8f3bd6-40a3-4249-9328-6c576f575779   
791  fed1458a-14a9-41af-b32a-1fda98e285e6   
792  ff1de9c2-7098-4d51-9e75-1d5990fcec55   
793  ff7d9fe9-619e-4641-9f25-b474545a3c6f   
794  ff931395-97ad-46b1-8add-ee6b47f4056f   

                                      measuring_points  
0    {'_id': '007ff213-9b5d-4243-9c8b-5eae997ac0ee'...  
1    {'_id': '0108e474-f1f4-4274-bf9f-e51d4963c091'...  
2    {'_id': '023794f2-5e3d-4450-8b71-8aae79d381a1'...  
3    {'_id': '0245cbf0-faec-4708-a1f2-3ccf7ebec1e1'...  
4    {'_id': '02498907-b394-4b7c-a0ab-068692746591'...  
..                                                 ...  
790  {'_id': 'fe8f3bd6-40a3-4249-9328-6c576f575779'...  
791